# Loop 42 LB Feedback Analysis

**CRITICAL**: exp_041 (Aggressive Regularization) submission results:
- CV: 0.0090
- LB: 0.0932
- CV-LB ratio: 10.36x

**Hypothesis REJECTED**: Aggressive regularization did NOT reduce the CV-LB gap.

Now we need to understand what's really happening and find a new path forward.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# All submission data
submissions = [
    {'exp': 'exp_000', 'cv': 0.0111, 'lb': 0.0982, 'name': 'Baseline MLP'},
    {'exp': 'exp_001', 'cv': 0.0123, 'lb': 0.1065, 'name': 'LightGBM'},
    {'exp': 'exp_003', 'cv': 0.0105, 'lb': 0.0972, 'name': 'Spange+DRFP'},
    {'exp': 'exp_005', 'cv': 0.0104, 'lb': 0.0969, 'name': 'Large Ensemble'},
    {'exp': 'exp_006', 'cv': 0.0097, 'lb': 0.0946, 'name': 'Simpler Model'},
    {'exp': 'exp_007', 'cv': 0.0093, 'lb': 0.0932, 'name': 'Even Simpler'},
    {'exp': 'exp_009', 'cv': 0.0092, 'lb': 0.0936, 'name': 'Ridge Regression'},
    {'exp': 'exp_012', 'cv': 0.0090, 'lb': 0.0913, 'name': 'Simple Ensemble'},
    {'exp': 'exp_024', 'cv': 0.0087, 'lb': 0.0893, 'name': 'ACS PCA Fixed'},
    {'exp': 'exp_026', 'cv': 0.0085, 'lb': 0.0887, 'name': 'Weighted Loss'},
    {'exp': 'exp_030', 'cv': 0.0083, 'lb': 0.0877, 'name': 'GP Ensemble (0.2)'},
    {'exp': 'exp_041', 'cv': 0.0090, 'lb': 0.0932, 'name': 'Aggressive Regularization'},
]

df = pd.DataFrame(submissions)
df['ratio'] = df['lb'] / df['cv']
df['gap'] = df['lb'] - df['cv']

print('=== ALL SUBMISSIONS ===')
print(df.to_string(index=False))
print(f'\nTarget: 0.0347')
print(f'Best LB: {df["lb"].min():.4f} (exp_030)')
print(f'Gap to target: {df["lb"].min() / 0.0347:.2f}x')

In [ ]:
# Analyze CV-LB relationship
from scipy.stats import linregress

cv = df['cv'].values
lb = df['lb'].values

slope, intercept, r_value, p_value, std_err = linregress(cv, lb)

print('=== CV-LB LINEAR REGRESSION ===')
print(f'LB = {slope:.2f} * CV + {intercept:.4f}')
print(f'R² = {r_value**2:.4f}')
print(f'\nInterpretation:')
print(f'  - Slope: {slope:.2f} (each 0.001 CV improvement gives {slope*0.001:.4f} LB improvement)')
print(f'  - Intercept: {intercept:.4f} (baseline LB when CV=0)')
print(f'\nTo reach target LB=0.0347:')
required_cv = (0.0347 - intercept) / slope
print(f'  - Required CV: {required_cv:.6f}')
if required_cv < 0:
    print(f'  - IMPOSSIBLE: Would require negative CV!')
    print(f'  - The intercept ({intercept:.4f}) is already above target ({0.0347})')
else:
    print(f'  - This is {required_cv / df["cv"].min():.2f}x better than best CV ({df["cv"].min():.4f})')

In [ ]:
# Plot CV vs LB with regression line
plt.figure(figsize=(10, 6))
plt.scatter(cv, lb, s=100, alpha=0.7, label='Submissions')

# Regression line
cv_range = np.linspace(0, 0.015, 100)
lb_pred = slope * cv_range + intercept
plt.plot(cv_range, lb_pred, 'r--', label=f'LB = {slope:.2f}*CV + {intercept:.4f}')

# Target line
plt.axhline(y=0.0347, color='g', linestyle=':', linewidth=2, label=f'Target: 0.0347')

# Best submission
best_idx = df['lb'].idxmin()
plt.scatter([df.loc[best_idx, 'cv']], [df.loc[best_idx, 'lb']], 
            s=200, marker='*', color='gold', edgecolors='black', linewidth=2,
            label=f'Best: exp_030 (LB={df.loc[best_idx, "lb"]:.4f})')

# Aggressive regularization (latest)
latest = df[df['exp'] == 'exp_041'].iloc[0]
plt.scatter([latest['cv']], [latest['lb']], 
            s=200, marker='X', color='red', edgecolors='black', linewidth=2,
            label=f'Aggressive Reg: exp_041 (LB={latest["lb"]:.4f})')

plt.xlabel('CV Score (MSE)', fontsize=12)
plt.ylabel('LB Score (MSE)', fontsize=12)
plt.title('CV vs LB Relationship - Aggressive Regularization FAILED', fontsize=14)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('/home/code/exploration/cv_lb_analysis_loop42.png', dpi=150)
plt.show()

print('\n=== KEY INSIGHT ===')
print('Aggressive regularization (exp_041) is ON THE SAME LINE as other submissions.')
print('This means the CV-LB gap is NOT due to overfitting - it\'s STRUCTURAL.')
print('The gap is likely due to how Kaggle evaluates submissions (different data, different seeds, etc.)')

In [ ]:
# Analyze the CV-LB ratio for each submission
print('=== CV-LB RATIO ANALYSIS ===')
df_sorted = df.sort_values('lb')
for _, row in df_sorted.iterrows():
    print(f"{row['exp']:8s} | CV: {row['cv']:.4f} | LB: {row['lb']:.4f} | Ratio: {row['ratio']:.2f}x | {row['name']}")

print(f'\nMean ratio: {df["ratio"].mean():.2f}x')
print(f'Std ratio: {df["ratio"].std():.2f}x')
print(f'Min ratio: {df["ratio"].min():.2f}x ({df.loc[df["ratio"].idxmin(), "exp"]})')
print(f'Max ratio: {df["ratio"].max():.2f}x ({df.loc[df["ratio"].idxmax(), "exp"]})')

In [ ]:
# What would it take to reach the target?
print('=== PATH TO TARGET ANALYSIS ===')
print(f'\nTarget: 0.0347')
print(f'Best LB: {df["lb"].min():.4f}')
print(f'Gap: {df["lb"].min() - 0.0347:.4f} ({(df["lb"].min() - 0.0347) / 0.0347 * 100:.1f}% above target)')

print('\n=== OPTION 1: Improve CV (current approach) ===')
print(f'Current CV-LB relationship: LB = {slope:.2f}*CV + {intercept:.4f}')
print(f'Intercept ({intercept:.4f}) > Target ({0.0347})')
print('IMPOSSIBLE: Even with CV=0, LB would be {:.4f}'.format(intercept))

print('\n=== OPTION 2: Change the CV-LB relationship ===')
print('Need to reduce the intercept from {:.4f} to below {:.4f}'.format(intercept, 0.0347))
print('This requires a FUNDAMENTALLY DIFFERENT approach:')
print('  1. Different model architecture (not just regularization)')
print('  2. Different feature engineering')
print('  3. Different training procedure')
print('  4. Domain adaptation techniques')

print('\n=== OPTION 3: Find what makes Kaggle evaluation different ===')
print('The CV-LB gap is consistent (~10x) across ALL submissions.')
print('This suggests the gap is due to:')
print('  1. Different random seeds in Kaggle evaluation')
print('  2. Different data ordering')
print('  3. Different train/test split')
print('  4. Hidden test data with different distribution')

In [ ]:
# Analyze what's special about exp_030 (best LB)
print('=== EXP_030 ANALYSIS (BEST LB) ===')
print('exp_030: GP Ensemble with GP weight 0.2')
print('  - CV: 0.0083')
print('  - LB: 0.0877')
print('  - Ratio: 10.57x')
print('\nWhat made it special?')
print('  - GP weight: 0.2 (vs 0.15 in other experiments)')
print('  - GP provides uncertainty estimates')
print('  - GP is more robust to distribution shift')
print('\nBut aggressive regularization (exp_041) did NOT improve LB:')
print('  - CV: 0.0090 (worse than exp_030)')
print('  - LB: 0.0932 (worse than exp_030)')
print('  - Ratio: 10.36x (similar to exp_030)')
print('\nConclusion: The CV-LB gap is NOT due to overfitting.')

In [ ]:
# What approaches haven't been tried?
print('=== UNEXPLORED APPROACHES ===')
print('\n1. PURE GP MODEL (no MLP, no LGBM)')
print('   - GP is known to generalize better to unseen data')
print('   - exp_032 tried pure GP but was not submitted')
print('   - Worth submitting to see if CV-LB relationship changes')

print('\n2. DIFFERENT FEATURE SETS')
print('   - Current: Spange + DRFP + ACS PCA + Arrhenius')
print('   - Try: Only Spange (simpler, more interpretable)')
print('   - Try: Only DRFP (molecular structure)')
print('   - Try: Only ACS PCA (chemical properties)')

print('\n3. DIFFERENT ENSEMBLE STRATEGIES')
print('   - Current: Weighted average of GP + MLP + LGBM')
print('   - Try: Stacking with meta-learner')
print('   - Try: Blending with different weights per fold')

print('\n4. DOMAIN ADAPTATION')
print('   - Train on single solvents, adapt to mixtures')
print('   - Use adversarial training to align distributions')

print('\n5. TRANSFER LEARNING')
print('   - Pre-train on related chemical data')
print('   - Fine-tune on catechol data')

In [ ]:
# Final recommendation
print('=== FINAL RECOMMENDATION ===')
print('\nThe aggressive regularization hypothesis was REJECTED.')
print('The CV-LB gap is STRUCTURAL, not due to overfitting.')
print('\nWith 4 submissions remaining, we should:')
print('\n1. SUBMIT exp_032 (Pure GP) to test if GP alone has better CV-LB ratio')
print('   - Pure GP might have different CV-LB relationship')
print('   - GP is known to generalize better to unseen data')
print('\n2. Try DIFFERENT FEATURE ENGINEERING')
print('   - The current features might not capture the right information')
print('   - Try domain-specific features (reaction mechanism, transition states)')
print('\n3. Try ENSEMBLE DIVERSITY')
print('   - Train models on different feature subsets')
print('   - Use stacking instead of simple averaging')
print('\n4. ACCEPT that target might be unreachable with current approach')
print('   - The intercept (0.0524) is 1.51x higher than target (0.0347)')
print('   - Need a fundamentally different approach')
print('\nBUT WE NEVER GIVE UP! The target IS reachable.')
print('We just need to find the RIGHT approach.')